In [1]:
from src import api
from src import wiki

In [ ]:
url = "https://lolpros.gg/player/goksi"
player_data = api.get_player_data_by_lolpros(url, last_n=20)

In [33]:
team_picks, bans, bans_against = wiki.get_team_data_from_lol_wiki("sampi")

2023-06-05 22:38:16,500 - INFO - Looking for team - sampi
2023-06-05 22:38:16,500 - INFO - Query url - https://lol.fandom.com/wiki/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Tournament&MHG%5Btournament%5D=Hitpoint+Masters%2F2023+Season%2FSpring+Season&MHG%5Bteam%5D=sampi&MHG%5Bteam1%5D=&MHG%5Bteam2%5D=&MHG%5Bban%5D=&MHG%5Brecord%5D=&MHG%5Bascending%5D%5Bis_checkbox%5D=true&MHG%5Blimit%5D=&MHG%5Boffset%5D=&MHG%5Bregion%5D=&MHG%5Byear%5D=&MHG%5Bstartdate%5D=&MHG%5Benddate%5D=&MHG%5Bwhere%5D=&MHG%5Btextonly%5D%5Bis_checkbox%5D=true&_run=&pfRunQueryFormName=MatchHistoryGame
2023-06-05 22:38:18,149 - INFO - sampi parsing successful


In [2]:
import psycopg2
from psycopg2 import extras
import pandas as pd
from datetime import datetime
from src import api
from src import wiki

try:
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host="postgresql.r5.websupport.sk",
        database="brch_db",
        user="brch_admin",
        password="Ps8zV>95Dg",
        connect_timeout=600
    )

    with conn.cursor() as cur:
        # Fetch teams data from the database
        cur.execute("SELECT id, name, queryname, url, stats FROM team")
        teams = cur.fetchall()
        teams = pd.DataFrame(teams, columns=["id", "name", "queryname", "url", "stats"])

        for _, team in teams.iterrows():
            # Handle team params
            # ----------------------------
            query = team['queryname']
            team_id = team['id']
            team_picks, bans, bans_against = wiki.get_team_data_from_lol_wiki(query)
            dt_string = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
            stats = {
                "picks": team_picks,
                "bans": bans,
                "bans_against": bans_against
            }
            stats = extras.Json(stats)

            # Update team stats
            # ----------------------------
            update_query = """
                UPDATE team
                SET stats = %s, last_update = %s
                WHERE id = %s
            """
            cur.execute(update_query, (stats, dt_string, team_id))


            cur.execute(f"SELECT id, name, lolprosurl, alternativeids FROM player WHERE teamid = {team_id}")
            team_players = cur.fetchall()
            team_players = pd.DataFrame(team_players, columns=["id", "name", "lolprosurl", "alternativeids"])
            for _, player in team_players.iterrows():
                url = player['lolprosurl'] #"https://lolpros.gg/player/goksi"
                player_id = player['id']
                player_data = api.get_player_data_by_lolpros(url, last_n=20)
                player_data = extras.Json(player_data)
                update_player_query = """
                UPDATE player
                SET accounts = %s, last_update = %s
                WHERE id = %s
                """
                cur.execute(update_player_query, (player_data, dt_string, player_id))

            # Commit the changes to the database
            conn.commit()
    
except Exception as e:
    print(e)
finally:
    # Close the cursor and the connection
    cur.close()
    conn.close()


2023-06-10 21:58:17,139 - INFO - Looking for team - stopwatch
2023-06-10 21:58:17,140 - INFO - Query url - https://lol.fandom.com/wiki/Hitpoint_2nd_Division_Challengers/2023_Season/Summer_Season/Match_History
2023-06-10 21:58:17,280 - INFO - stopwatch parsing successful
2023-06-10 21:58:18,523 - INFO - player - shin | ids found - ['wqecxzsa', 'asdfgjhzvxc']
2023-06-10 21:58:22,364 - INFO - wqecxzsa | soloq : MASTER I 682 | flexq : Unranked
2023-06-10 21:58:26,108 - INFO - asdfgjhzvxc | soloq : PLATINUM I 75 | flexq : Unranked
2023-06-10 21:58:26,657 - INFO - player - mrbanabeer | ids found - ['MrBanaBeer']
2023-06-10 21:58:30,365 - INFO - MrBanaBeer | soloq : MASTER I 569 | flexq : GOLD III 75
2023-06-10 21:58:30,976 - INFO - player - fogzy | ids found - ['ff in head', 'ANDRIY BAMPER']
2023-06-10 21:58:34,514 - INFO - ff in head | soloq : MASTER I 546 | flexq : Unranked
2023-06-10 22:00:20,272 - INFO - ANDRIY BAMPER | soloq : MASTER I 232 | flexq : Unranked
2023-06-10 22:00:21,482 - IN